# Convolutional Attention-based Seq2Seq neural network for end-to-end ASR

## define the phoneme labels, paths, hyperparameters

In [1]:
phn_61 = ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay', 'b', 'bcl', 'ch', 'd', 'dcl', 'dh', 'dx', 'eh', 'el', 'em', 'en', 'eng', 'epi', 'er', 'ey', 'f', 'g', 'gcl', 'h#', 'hh', 'hv', 'ih', 'ix', 'iy', 'jh', 'k', 'kcl', 'l', 'm', 'n', 'ng', 'nx', 'ow', 'oy', 'p', 'pau', 'pcl', 'q', 'r', 's', 'sh', 't', 'tcl', 'th', 'uh', 'uw', 'ux', 'v', 'w', 'y', 'z', 'zh']
phn_39 = ['ae', 'ao', 'aw', 'ax', 'ay', 'b', 'ch', 'd', 'dh', 'dx', 'eh', 'er', 'ey', 'f', 'g', 'h#', 'hh', 'ix', 'iy', 'jh', 'k', 'l', 'm', 'n', 'ng', 'ow', 'oy', 'p', 'r', 's', 't', 'th', 'uh', 'uw', 'v', 'w', 'y', 'z', 'zh']
mapping = {'ah': 'ax', 'ax-h': 'ax', 'ux': 'uw', 'aa': 'ao', 'ih': 'ix', 'axr': 'er', 'el': 'l', 'em': 'm', 'en': 'n', 'nx': 'n', 'eng': 'ng', 'sh': 'zh', 'hv': 'hh', 'bcl': 'h#', 'pcl': 'h#', 'dcl': 'h#', 'tcl': 'h#', 'gcl': 'h#', 'kcl': 'h#', 'q': 'h#', 'epi': 'h#', 'pau': 'h#'}

TRAIN_FILE = './data/fbank/train.tfrecords'
DEV_FILE = './data/fbank/dev.tfrecords'
TEST_FILE = './data/fbank/test.tfrecords'
checkpoints_path = './model/ckpt'

feat_type = 'fbank'
feats_dim = 39 if feat_type=='mfcc' else 123
labels_sos_id = len(phn_61) + 1
labels_eos_id = len(phn_61) 
num_classes = len(phn_61) + 1

num_encoder = 256
num_decoder = 256
keep_prob = 0.5
n_encoder_layer = 3
beam_width = 10
batch_size = 32

In [2]:
import tensorflow as tf
from tensorflow.python.layers import core as layers_core
import numpy as np
import os, time
from tensorflow.python.ops import rnn_cell_impl
from tensorflow.python.util import nest

## modify existing tensorflow code
- apply dropout to attentional layer
- apply tanh activation to attentional layer

In [3]:
def _compute_attention(attention_mechanism, cell_output, previous_alignments,
                       attention_layer, attention_dropout_layer, training):
    """Computes the attention and alignments for a given attention_mechanism."""
    alignments = attention_mechanism(cell_output, previous_alignments=previous_alignments)

    expanded_alignments = tf.expand_dims(alignments, 1)
    context = tf.matmul(expanded_alignments, attention_mechanism.values)
    context = tf.squeeze(context, [1])

    if attention_layer is not None:
        attention = attention_dropout_layer(attention_layer(tf.concat([cell_output, context], 1)), training=training)
    else:
        attention = context

    return attention, alignments    
    
class MyAttentionWrapper(tf.contrib.seq2seq.AttentionWrapper):
    
    def __init__(self, cell, attention_mechanism, keep_prob, training, attention_layer_size=None, alignment_history=False, cell_input_fn=None,
                output_attention=True, initial_cell_state=None, name=None):
        super(MyAttentionWrapper, self).__init__(cell, attention_mechanism, attention_layer_size, alignment_history, cell_input_fn,
                output_attention, initial_cell_state, name)
        
        self.keep_prob = keep_prob
        self.training = training
        
        super(tf.contrib.seq2seq.AttentionWrapper, self).__init__(name=name)
        if not rnn_cell_impl._like_rnncell(cell):  # pylint: disable=protected-access
            raise TypeError("cell must be an RNNCell, saw type: %s" % type(cell).__name__)
        if isinstance(attention_mechanism, (list, tuple)):
            self._is_multi = True
            attention_mechanisms = attention_mechanism
            for attention_mechanism in attention_mechanisms:
                if not isinstance(attention_mechanism, AttentionMechanism):
                    raise TypeError("attention_mechanism must contain only instances of "
                                    "AttentionMechanism, saw type: %s" % type(attention_mechanism).__name__)
        else:
            self._is_multi = False
            if not isinstance(attention_mechanism, tf.contrib.seq2seq.AttentionMechanism):
                raise TypeError("attention_mechanism must be an AttentionMechanism or list of "
                                "multiple AttentionMechanism instances, saw type: %s" % type(attention_mechanism).__name__)
            attention_mechanisms = (attention_mechanism,)

        if cell_input_fn is None:
            cell_input_fn = (lambda inputs, attention: tf.concat([inputs, attention], -1))
        else:
            if not callable(cell_input_fn):
                raise TypeError("cell_input_fn must be callable, saw type: %s" % type(cell_input_fn).__name__)

        if attention_layer_size is not None:
            attention_layer_sizes = tuple(attention_layer_size if isinstance(attention_layer_size, (list, tuple))
                                              else (attention_layer_size,))
            if len(attention_layer_sizes) != len(attention_mechanisms):
                raise ValueError("If provided, attention_layer_size must contain exactly one "
                                "integer per attention_mechanism, saw: %d vs %d" % (len(attention_layer_sizes), len(attention_mechanisms)))
            self._attention_layers = tuple(layers_core.Dense(attention_layer_size, name="attention_layer", use_bias=True,
                    activation=tf.tanh) for attention_layer_size in attention_layer_sizes)
            self._attention_dropout_layers = tuple(layers_core.Dropout(rate=1-self.keep_prob, name="attention_dropout_layer")
                                                  for attention_layer_size in attention_layer_sizes)
            self._attention_layer_size = sum(attention_layer_sizes)
        else:
            self._attention_layers = None
            self._attention_dropout_layers = None
            self._attention_layer_size = sum(attention_mechanism.values.get_shape()[-1].value
                                                      for attention_mechanism in attention_mechanisms)
            
        self._cell = cell
        self._attention_mechanisms = attention_mechanisms
        self._cell_input_fn = cell_input_fn
        self._output_attention = output_attention
        self._alignment_history = alignment_history
        with tf.name_scope(name, "AttentionWrapperInit"):
            if initial_cell_state is None:
                self._initial_cell_state = None
            else:
                final_state_tensor = nest.flatten(initial_cell_state)[-1]
                state_batch_size = (final_state_tensor.shape[0].value or tf.shape(final_state_tensor)[0])
                error_message = (
                "When constructing AttentionWrapper %s: " % self._base_name +
                "Non-matching batch sizes between the memory "
                "(encoder output) and initial_cell_state.  Are you using "
                "the BeamSearchDecoder?  You may need to tile your initial state "
                "via the tf.contrib.seq2seq.tile_batch function with argument "
                "multiple=beam_width.")
                with tf.control_dependencies(
                    self._batch_size_checks(state_batch_size, error_message)):
                    self._initial_cell_state = nest.map_structure(lambda s: tf.identity(s, name="check_initial_cell_state"),
                                                              initial_cell_state)
                    
    def call(self, inputs, state):

        if not isinstance(state, tf.contrib.seq2seq.AttentionWrapperState):
            raise TypeError("Expected state to be instance of MyAttentionWrapperState. "
                          "Received type %s instead."  % type(state))
        
        cell_inputs = self._cell_input_fn(inputs, state.attention)
        cell_state = state.cell_state
        cell_output, next_cell_state = self._cell(cell_inputs, cell_state)

        cell_batch_size = (
            cell_output.shape[0].value or tf.shape(cell_output)[0])
        error_message = (
            "When applying AttentionWrapper %s: " % self.name +
            "Non-matching batch sizes between the memory "
            "(encoder output) and the query (decoder output).  Are you using "
            "the BeamSearchDecoder?  You may need to tile your memory input via "
            "the tf.contrib.seq2seq.tile_batch function with argument "
            "multiple=beam_width.")
        with tf.control_dependencies(
            self._batch_size_checks(cell_batch_size, error_message)):
            cell_output = tf.identity(
              cell_output, name="checked_cell_output")

        if self._is_multi:
            previous_alignments = state.alignments
            previous_alignment_history = state.alignment_history
        else:
            previous_alignments = [state.alignments]
            previous_alignment_history = [state.alignment_history]

        all_alignments = []
        all_attentions = []
        all_histories = []
        for i, attention_mechanism in enumerate(self._attention_mechanisms):
            attention, alignments = _compute_attention(
            attention_mechanism, cell_output, previous_alignments[i],
            self._attention_layers[i] if self._attention_layers else None,
            self._attention_dropout_layers[i] if self._attention_dropout_layers else None,
            self.training)
            alignment_history = previous_alignment_history[i].write(
            state.time, alignments) if self._alignment_history else ()

            all_alignments.append(alignments)
            all_histories.append(alignment_history)
            all_attentions.append(attention)

        attention = tf.concat(all_attentions, 1)
        
        next_state = tf.contrib.seq2seq.AttentionWrapperState(
            time=state.time + 1,
            cell_state=next_cell_state,
            attention=attention,
            alignments=self._item_or_tuple(all_alignments),
            alignment_history=self._item_or_tuple(all_histories))
        
        if self._output_attention:
            return attention, next_state
        else:
            return cell_output, next_state

## model definitions

In [4]:
class Model(object):
    def __init__(self, data_path, model_type):
        self.data_path = data_path
        self.model_type = model_type

        self._get_data()
        self._build()
    
    def _get_data(self):
        iterator = self._get_iterator()
        self.iterator_initializer = iterator.initializer
        
        batched_data = iterator.get_next()
        self.features = batched_data[0]
        self.labels_in = batched_data[1]
        self.labels_out = batched_data[2]
        self.feats_seq_len = tf.to_int32(batched_data[3])
        self.labels_in_seq_len = tf.to_int32(batched_data[4])
        
    def _build(self):
        self.keep_prob = tf.placeholder(tf.float32)
        self.training = tf.placeholder(tf.bool)
    
        decoded = self._compute_dynamic_decode()
        self.saver = tf.train.Saver(max_to_keep=5)
        
        if self.model_type == 'train':
            self.logits = decoded
            self.loss = self._compute_loss()
            self.update_step = self._get_update_step()
        elif self.model_type == 'eval':
            self.logits = decoded
            self.loss = self._compute_loss()
        elif self.model_type == 'infer':
            self.predicted_ids = decoded
            self.per = self._compute_per() # tuple of unnormalized edit distance and seqeucne_length
        else:
            raise Exception('invalid model_type')
        
    def _get_iterator(self):
        dataset = tf.contrib.data.TFRecordDataset(self.data_path)
        context_features = {'feats_seq_len': tf.FixedLenFeature([], dtype=tf.int64),
                           'labels_seq_len': tf.FixedLenFeature([], dtype=tf.int64)}
        sequence_features = {'features': tf.FixedLenSequenceFeature([feats_dim], dtype=tf.float32),
                            'labels': tf.FixedLenSequenceFeature([], dtype=tf.int64)}
        dataset = dataset.map(lambda serialized_example: tf.parse_single_sequence_example(serialized_example,
                                                                                         context_features=context_features,
                                                                                         sequence_features=sequence_features))
        dataset = dataset.map(lambda context, sequence: (sequence['features'], sequence['labels'], 
                                                         context['feats_seq_len'], context['labels_seq_len']))
        dataset = dataset.map(lambda features, labels, feats_seq_len, labels_seq_len: (features, 
                                                        tf.concat(([labels_sos_id], labels),0),
                                                        tf.concat((labels, [labels_eos_id]), 0),
                                                                feats_seq_len, labels_seq_len))
        dataset = dataset.map(lambda features, labels_in, labels_out, feats_seq_len, labels_seq_len: 
                                          (features, labels_in, labels_out, feats_seq_len, tf.size(labels_in, out_type=tf.int64)))
        def batching_func(x):
            return x.padded_batch(batch_size,
                                 padded_shapes=(tf.TensorShape([None, feats_dim]),
                                               tf.TensorShape([None]),
                                               tf.TensorShape([None]),
                                               tf.TensorShape([]),
                                               tf.TensorShape([])),
                                 padding_values=(tf.cast(0, tf.float32),
                                                tf.cast(labels_eos_id, tf.int64),
                                                tf.cast(labels_eos_id, tf.int64),
                                                tf.cast(0, tf.int64),
                                                tf.cast(0, tf.int64)))
        
        def key_func(features, labels_in, labels_out, feats_seq_len, labels_in_seq_len):
            f0 = lambda: tf.constant(0, tf.int64)
            f1 = lambda: tf.constant(1, tf.int64)
            f2 = lambda: tf.constant(2, tf.int64)
            f3 = lambda: tf.constant(3, tf.int64)
            f4 = lambda: tf.constant(4, tf.int64)
            f5 = lambda: tf.constant(5, tf.int64)
            f6 = lambda: tf.constant(6, tf.int64)
            
            return tf.case([(tf.less_equal(feats_seq_len, 200), f0),
                   (tf.less_equal(feats_seq_len, 250), f1),
                   (tf.less_equal(feats_seq_len, 300), f2),
                   (tf.less_equal(feats_seq_len, 350), f3),
                   (tf.less_equal(feats_seq_len, 400), f4),
                   (tf.less_equal(feats_seq_len, 500), f5)], default=f6)
        
        def reduce_func(bucket_id, windowed_data):
            return batching_func(windowed_data)
        
        if self.model_type=='train':
            self.shuffle_seed = tf.placeholder(tf.int64)
            dataset = dataset.shuffle(10000, seed=self.shuffle_seed)
            batched_dataset = dataset.group_by_window(key_func=key_func, reduce_func=reduce_func, window_size=batch_size)
            batched_dataset = batched_dataset.shuffle(10000, seed=-self.shuffle_seed)
        else: # eval or infer, no shuffle
            batched_dataset = batching_func(dataset)
            
        return batched_dataset.make_initializable_iterator()
    
    def _compute_encoder_outputs(self):
        def residual_block(inp, out_channels):
            inp_channels = inp.get_shape().as_list()[3]
            out = tf.layers.conv2d(inp, filters=out_channels, kernel_size=(3,3), strides=(1,1), padding='same', use_bias=False)
            out = tf.layers.batch_normalization(out, training=self.training) 
            out = tf.nn.relu(out)
            out = tf.layers.dropout(out, rate=1-self.keep_prob, training=self.training)
            out = tf.layers.conv2d(out, filters=out_channels, kernel_size=(3,3), strides=(1,1), padding='same', use_bias=False)
            out = tf.layers.batch_normalization(out, training=self.training)
            out = tf.nn.relu(out)
            out = tf.layers.dropout(out, rate=1-self.keep_prob, training=self.training)
            if inp_channels != out_channels:
                inp = tf.layers.conv2d(inp, filters=out_channels, kernel_size=(1,1), strides=(1,1), padding='same')
            return out + inp
        def conv_block(inp, filters, kernel_size, strides):
            out = tf.layers.conv2d(inp, filters=filters, kernel_size=kernel_size, strides=strides, padding='same', use_bias=False)
            out = tf.layers.batch_normalization(out, training=self.training) 
            out = tf.nn.relu(out)
            out = tf.layers.dropout(out, rate=1-self.keep_prob, training=self.training)
            return out
        def dense_block(inp, num_units):
            out = tf.layers.dense(inp, num_units, use_bias=False)
            out = tf.layers.batch_normalization(out, training=self.training) 
            out = tf.nn.relu(out)
            out = tf.layers.dropout(out, rate=1-self.keep_prob, training=self.training)
            return out
         
        features = tf.stack(tf.split(self.features, num_or_size_splits=3, axis=2), axis=3)
        features = tf.transpose(features, [0,2,1,3]) # shape = [batch, feats_dim/3, max_time, channels]
        
        conv = conv_block(features, 128, (3,3), (1,3)) # time -> time/3
        conv = residual_block(conv, 64)
        conv = residual_block(conv, 64)
        conv = residual_block(conv, 64)
        
        flattend = tf.transpose(conv, [0,2,1,3])
        flattend = tf.reshape(flattend, [tf.shape(flattend)[0], tf.shape(flattend)[1], 41*64])
        
        fc = dense_block(flattend, 1024)
        
        cells_fw = [tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_encoder, use_peepholes=True, initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1)),
                                                  output_keep_prob=self.keep_prob) for _ in range(n_encoder_layer)]
        cells_bw = [tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_encoder, use_peepholes=True, initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1)),
                                                  output_keep_prob=self.keep_prob) for _ in range(n_encoder_layer)]
        
        outputs, _, _ = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, fc,
                                                            dtype=tf.float32, sequence_length=self.feats_seq_len//3)
        return outputs, self.feats_seq_len//3
        
    def _get_decoder_cell_and_init_state(self):    
        num_batch = tf.shape(self.memory)[0]
        decoder_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(num_decoder, use_peepholes=True, initializer=tf.random_uniform_initializer(minval=-0.1, maxval=0.1)),
                                                     output_keep_prob=self.keep_prob)

        if self.model_type == 'infer':
            self.memory = tf.contrib.seq2seq.tile_batch(self.memory, multiplier=beam_width)
            self.mem_seq_len = tf.contrib.seq2seq.tile_batch(self.mem_seq_len, multiplier=beam_width)
            num_batch = num_batch * beam_width

        attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_decoder, self.memory, self.mem_seq_len, scale=False)
        decoder_cell = MyAttentionWrapper(decoder_cell, attention_mechanism, self.keep_prob, self.training,
                                          attention_layer_size=num_decoder, output_attention=True)
        decoder_initial_state = decoder_cell.zero_state(num_batch, tf.float32)
        return (decoder_cell, decoder_initial_state)
    
    def _compute_dynamic_decode(self):
        embedding_decoder = tf.Variable(tf.concat([np.identity(num_classes-1), tf.zeros([2,num_classes-1])], axis=0),
                                        dtype=tf.float32, trainable=False)
        #embedding_decoder = tf.Variable(tf.random_uniform([num_classes+1, 30], minval=-0.05, maxval=0.05, dtype=tf.float32))
        decoder_emb_inp = tf.nn.embedding_lookup(embedding_decoder, self.labels_in)
    
        self.memory, self.mem_seq_len = self._compute_encoder_outputs()
        with tf.variable_scope('decoder') as decoder_scope:
            decoder_cell, decoder_initial_state = self._get_decoder_cell_and_init_state()
        
            output_layer = layers_core.Dense(num_classes, use_bias=True)

            if self.model_type in ['train', 'eval']:
                helper = tf.contrib.seq2seq.TrainingHelper(decoder_emb_inp, self.labels_in_seq_len, time_major=False)
                decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state=decoder_initial_state)
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major=False, scope=decoder_scope)
                return output_layer(outputs.rnn_output)
            else: # infer
                beam_decoder = tf.contrib.seq2seq.BeamSearchDecoder(decoder_cell, embedding_decoder, 
                                                    tf.fill([tf.shape(self.features)[0]], labels_sos_id), labels_eos_id,
                                                            decoder_initial_state, beam_width, output_layer=output_layer)
                decoded, _, final_seq_len = tf.contrib.seq2seq.dynamic_decode(beam_decoder, maximum_iterations=100,
                                                                              output_time_major=False, scope=decoder_scope)
                return decoded.predicted_ids # shape = [batch, max_time, beam_width]
    
    def _compute_loss(self):
        max_time = tf.shape(self.labels_out)[1]
        target_weights = tf.sequence_mask(self.labels_in_seq_len, max_time, dtype=self.logits.dtype)
        crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=self.labels_out, logits=self.logits)
        return tf.reduce_sum(crossent * target_weights) / tf.to_float(tf.shape(self.logits)[0])
    
    def _get_update_step(self):
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            params = tf.trainable_variables()
            #
            if weight_decay is not None:
                l2_loss = 0
                for v in tf.trainable_variables():
                    if 'kernel' in v.name:
                        l2_loss += tf.nn.l2_loss(v)
                self.loss += weight_decay * l2_loss
            #
            gradients = tf.gradients(self.loss, params)
            clipped_gradients, _ = tf.clip_by_global_norm(gradients, 1.0)
            #clipped_gradients = gradients # no clipping
            optimizer = tf.train.AdamOptimizer(learning_rate)
            update_step = optimizer.apply_gradients(zip(clipped_gradients, params))
        return update_step
    
    def _get_sparse_tensor(self, dense, default):
        indices = tf.to_int64(tf.where(tf.not_equal(dense, default)))
        vals = tf.to_int32(tf.gather_nd(dense, indices))
        shape = tf.to_int64(tf.shape(dense))
        return tf.SparseTensor(indices, vals, shape)
    
    def _compute_per(self):
        '''
        return tuple: (unnormalized edit distance, seqeucne_length),
        it is just sum of batched data
        '''
        phn_61_tensor = tf.constant(phn_61, dtype=tf.string)
        phn_39_tensor = tf.constant(phn_39, dtype=tf.string)
        mapping_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(mapping.keys()), list(mapping.values())), default_value='')
        self.mapping_table_init = mapping_table.init
        
        def map_to_reduced_phn(p):
            val = mapping_table.lookup(phn_61_tensor[p])
            f1 = lambda: tf.to_int32(tf.reduce_min(tf.where(tf.equal(val, phn_39_tensor))))
            f2 = lambda: tf.to_int32(tf.reduce_min(tf.where(tf.equal(phn_61_tensor[p], phn_39_tensor))))
            return tf.cond(tf.not_equal(val, ''), f1, f2)
        
        indices = tf.to_int64(tf.where(tf.logical_and(tf.not_equal(self.predicted_ids[:,:,0], -1), 
                                                      tf.not_equal(self.predicted_ids[:,:,0], labels_eos_id))))
        vals = tf.to_int32(tf.gather_nd(self.predicted_ids[:,:,0], indices))
        shape = tf.to_int64(tf.shape(self.predicted_ids[:,:,0]))
        decoded_sparse = tf.SparseTensor(indices, vals, shape)
        labels_out_sparse = self._get_sparse_tensor(self.labels_out, labels_eos_id)
        
        decoded_reduced = tf.SparseTensor(decoded_sparse.indices, tf.map_fn(map_to_reduced_phn, decoded_sparse.values), decoded_sparse.dense_shape)
        labels_out_reduced = tf.SparseTensor(labels_out_sparse.indices, tf.map_fn(map_to_reduced_phn, labels_out_sparse.values), labels_out_sparse.dense_shape)
        
        return tf.reduce_sum(tf.edit_distance(decoded_reduced, labels_out_reduced, normalize=False)) , tf.to_float(tf.size(labels_out_reduced.values))

## train model

In [5]:
learning_rate = 0.001
weight_decay = None
epochs = 50

train_graph = tf.Graph()
dev_graph = tf.Graph()

with train_graph.as_default():
    tf.set_random_seed(159)
    train_model = Model(data_path=TRAIN_FILE, model_type='train')
    initializer = tf.global_variables_initializer()

with dev_graph.as_default():
    dev_model = Model(data_path=DEV_FILE, model_type='eval')
    
train_sess = tf.Session(graph=train_graph)
train_sess.run(initializer)

dev_sess = tf.Session(graph=dev_graph)

for epoch in range(epochs):
    train_sess.run(train_model.iterator_initializer, feed_dict={train_model.shuffle_seed: epoch})
    train_loss = []
    start = time.time()
    while True:
        try:
            _, loss = train_sess.run([train_model.update_step, train_model.loss], feed_dict={train_model.keep_prob: keep_prob,
                                                                                             train_model.training: True})
            train_loss.append(loss)
        except tf.errors.OutOfRangeError:
            end = time.time()
            log = "Epoch {}/{}, \ntrain_loss={:.3f}, time={:.0f}s"
            print(log.format(epoch+1, epochs, np.mean(train_loss), end-start))
            if not os.path.isdir(os.path.split(checkpoints_path)[0]):
                os.makedirs(os.path.split(checkpoints_path)[0])
            saved_ckpt_path = train_model.saver.save(train_sess, checkpoints_path, global_step=epoch+1)
            
            dev_model.saver.restore(dev_sess, saved_ckpt_path)
            dev_sess.run(dev_model.iterator_initializer)
            start = time.time()
            dev_loss = []
            while True:
                try:
                    loss = dev_sess.run(dev_model.loss, feed_dict={dev_model.keep_prob: 1.0, dev_model.training: False})
                    dev_loss.append(loss)
                except tf.errors.OutOfRangeError:
                    end = time.time()
                    log = "\tdev_loss={:.3f}, time={:.0f}s"
                    print(log.format(np.mean(dev_loss), end-start))
                    break
            
            # go to next training epoch
            break
            
train_sess.close()
dev_sess.close()

Epoch 1/50, 
train_loss=150.404, time=145s
INFO:tensorflow:Restoring parameters from ./model/ckpt-1
	dev_loss=138.242, time=10s
Epoch 2/50, 
train_loss=125.022, time=136s
INFO:tensorflow:Restoring parameters from ./model/ckpt-2
	dev_loss=111.674, time=9s
Epoch 3/50, 
train_loss=113.847, time=133s
INFO:tensorflow:Restoring parameters from ./model/ckpt-3
	dev_loss=108.540, time=9s
Epoch 4/50, 
train_loss=111.071, time=134s
INFO:tensorflow:Restoring parameters from ./model/ckpt-4
	dev_loss=106.417, time=9s
Epoch 5/50, 
train_loss=108.955, time=133s
INFO:tensorflow:Restoring parameters from ./model/ckpt-5
	dev_loss=103.836, time=9s
Epoch 6/50, 
train_loss=106.841, time=134s
INFO:tensorflow:Restoring parameters from ./model/ckpt-6
	dev_loss=103.209, time=9s
Epoch 7/50, 
train_loss=105.354, time=134s
INFO:tensorflow:Restoring parameters from ./model/ckpt-7
	dev_loss=100.979, time=8s
Epoch 8/50, 
train_loss=103.998, time=133s
INFO:tensorflow:Restoring parameters from ./model/ckpt-8
	dev_loss=

KeyboardInterrupt: 

## fine tunning stage

In [9]:
learning_rate = 0.0001 # finetuning learning rate
weight_decay = 0.00001
epochs = 15

train_graph = tf.Graph()
dev_graph = tf.Graph()
test_graph = tf.Graph()

with train_graph.as_default():
    tf.set_random_seed(159)
    train_model = Model(data_path=TRAIN_FILE, model_type='train')
    initializer = tf.global_variables_initializer()

with dev_graph.as_default():
    dev_model = Model(data_path=DEV_FILE, model_type='eval')
with test_graph.as_default():
    test_model = Model(data_path=TEST_FILE, model_type='infer')
    
train_sess = tf.Session(graph=train_graph)
train_sess.run(initializer)
train_model.saver.restore(train_sess, './model/ckpt-46')

dev_sess = tf.Session(graph=dev_graph)

test_sess = tf.Session(graph=test_graph)
test_sess.run(test_model.mapping_table_init)

for epoch in range(epochs):
    train_sess.run(train_model.iterator_initializer, feed_dict={train_model.shuffle_seed: epoch})
    train_loss = []
    start = time.time()
    while True:
        try:
            _, loss = train_sess.run([train_model.update_step, train_model.loss], feed_dict={train_model.keep_prob: keep_prob,
                                                                                             train_model.training: True})
            train_loss.append(loss)
        except tf.errors.OutOfRangeError:
            end = time.time()
            log = "Epoch {}/{}, \ntrain_loss={:.3f}, time={:.0f}s"
            print(log.format(epoch+1, epochs, np.mean(train_loss), end-start))
            if not os.path.isdir(checkpoints_path):
                os.makedirs(checkpoints_path)
            saved_ckpt_path = train_model.saver.save(train_sess, checkpoints_path, global_step=epoch+1)
            
            dev_model.saver.restore(dev_sess, saved_ckpt_path)
            dev_sess.run(dev_model.iterator_initializer)
            start = time.time()
            dev_loss = []
            while True:
                try:
                    loss = dev_sess.run(dev_model.loss, feed_dict={dev_model.keep_prob: 1.0, dev_model.training: False})
                    dev_loss.append(loss)
                except tf.errors.OutOfRangeError:
                    end = time.time()
                    log = "\tdev_loss={:.3f}, time={:.0f}s"
                    print(log.format(np.mean(dev_loss), end-start))
                    break
            
            test_model.saver.restore(test_sess, saved_ckpt_path)
            test_sess.run(test_model.iterator_initializer)
            test_unnormed_edit_dist = []
            test_seq_len = []
            start = time.time()
            while True:
                try:
                    unnormed_edit_dist, seq_len = test_sess.run(test_model.per, feed_dict={test_model.keep_prob: 1.0,
                                                                                           test_model.training: False})
                    test_unnormed_edit_dist.append(unnormed_edit_dist)
                    test_seq_len.append(seq_len)
                except tf.errors.OutOfRangeError:
                    end = time.time()
                    log = '\t\ttest_per={:.3f}, time={:.0f}s'
                    print(log.format(sum(test_unnormed_edit_dist)/sum(test_seq_len), end-start))
                    break
            # go to next training epoch
            break
            
train_sess.close()
dev_sess.close()
test_sess.close()

INFO:tensorflow:Restoring parameters from ./model/ckpt-46
Epoch 1/15, 
train_loss=19.550, time=137s
INFO:tensorflow:Restoring parameters from ./model/ckpt-1
	dev_loss=23.258, time=9s
INFO:tensorflow:Restoring parameters from ./model/ckpt-1
		test_per=0.162, time=13s
Epoch 2/15, 
train_loss=18.846, time=137s
INFO:tensorflow:Restoring parameters from ./model/ckpt-2
	dev_loss=23.238, time=9s
INFO:tensorflow:Restoring parameters from ./model/ckpt-2
		test_per=0.162, time=12s
Epoch 3/15, 
train_loss=18.560, time=139s
INFO:tensorflow:Restoring parameters from ./model/ckpt-3
	dev_loss=23.272, time=9s
INFO:tensorflow:Restoring parameters from ./model/ckpt-3
		test_per=0.163, time=12s
Epoch 4/15, 
train_loss=18.233, time=137s
INFO:tensorflow:Restoring parameters from ./model/ckpt-4
	dev_loss=23.388, time=9s
INFO:tensorflow:Restoring parameters from ./model/ckpt-4
		test_per=0.164, time=12s
Epoch 5/15, 
train_loss=18.014, time=138s
INFO:tensorflow:Restoring parameters from ./model/ckpt-5
	dev_lo